In [ ]:
from sklearn.decomposition import PCA
import time
def pca(df_X, df_y):
    pca = PCA(n_components = 2)
    pca.fit(df_X)
    df_pca = pca.transform(df_X)
    df_pca = pd.DataFrame(df_pca, columns = ['comp. 0', 'comp. 1'])
    df_pca['target'] = df_y
    print('variance ratio:', pca.explained_variance_ratio_, 'sum:', sum(pca.explained_variance_ratio_))
    return df_pca

def concath(df_X, df_y):
    df = pd.concat([df_X, df_y])
    return df

In [ ]:
import tensorflow as tf
import math
pi = tf.constant(math.pi)

acc = lambda TN,FP,FN,TP: tf.math.divide_no_nan(TP+TN,TN+FP+FN+TP)
prec = lambda TN,FP,FN,TP: tf.math.divide_no_nan(TP,FP+TP)
rec = lambda TN,FP,FN,TP: tf.math.divide_no_nan(TP,FN+TP)
spec = lambda TN,FP,FN,TP: tf.math.divide_no_nan(TN,FP+TN)
f1s = lambda TN,FP,FN,TP: 2.*tf.math.divide_no_nan(prec(TN,FP,FN,TP)*\
                          rec(TN,FP,FN,TP),prec(TN,FP,FN,TP)+rec(TN,FP,FN,TP))
tss = lambda TN,FP,FN,TP: rec(TN,FP,FN,TP)+spec(TN,FP,FN,TP)-1.
csi = lambda TN,FP,FN,TP: tf.math.divide_no_nan(TP,FN+FP+TP)
hss1 = lambda TN,FP,FN,TP: tf.math.divide_no_nan(TP-FP,FN+TP)
hss2 = lambda TN,FP,FN,TP: tf.math.divide_no_nan(2.*((TP*TN)-(FP*FN)),((TP+FN)\
                            *(FN+TN))+((TP+FP)*(TN+FP)))
bacc = lambda TN,FP,FN,TP: tf.math.divide_no_nan(rec(TN,FP,FN,TP)+spec(TN,FP,FN,TP),2.0)

F_unif = lambda x: x
F_cos = lambda x,mu,delta: tf.where(x<mu-delta,0.,tf.where(x>mu+delta,1.,\
        0.5*(1.+tf.math.divide(x-mu,delta)+1./pi*tf.math.sin(pi*\
         tf.math.divide(x-mu,delta)))))



def SOL(score = 'accuracy', distribution = 'uniform',\
        mu = 0.5, delta = 0.1, mode = 'average'):

    if score == 'accuracy':
        score = acc
    if score == 'precision':
        score = prec
    if score == 'recall':
        score = rec
    if score == 'specificity':
        score = spec
    if score == 'f1_score':
        score = f1s
    if score == 'tss':
        score = tss
    if score == 'csi':
        score = csi
    if score == 'hss1':
        score = hss1
    if score == 'hss2':
        score = hss2
    if score == 'bacc':
        score = bacc

    if distribution == 'uniform':
        distr = F_unif

    if distribution == 'cosine':
        if type(mu) is not list:
            distr = lambda x: F_cos(x,mu,delta)
        else:
            distr = [lambda x: F_cos(x,mu[j],delta[j]) for j in \
                     range(0,len(mu))]

    def SOL_(y_true, y_pred):

        y_true = tf.convert_to_tensor(y_true, tf.float32)
        y_pred = tf.convert_to_tensor(y_pred, tf.float32)

        if y_pred.shape[1] == 1:

            TN = tf.reduce_sum((1.-y_true)*(1.-distr(y_pred)))
            TP = tf.reduce_sum(y_true*distr(y_pred))
            FP = tf.reduce_sum((1.-y_true)*distr(y_pred))
            FN = tf.reduce_sum(y_true*(1.-distr(y_pred)))

            return -score(TN,FP,FN,TP) + 1.

        else:

            score_arr = []
            num_c0_arr = []


            for j in range(0,y_pred.shape[1]):

                y_pred_ = y_pred[:,j]
                y_true_ = y_true[:,j]

                if type(mu) is not list:

                    TN = tf.reduce_sum((1.-y_true_)*(1.-distr(y_pred_)))
                    TP = tf.reduce_sum(y_true_*distr(y_pred_))
                    FP = tf.reduce_sum((1.-y_true_)*distr(y_pred_))
                    FN = tf.reduce_sum(y_true_*(1.-distr(y_pred_)))

                else:

                    TN = tf.reduce_sum((1.-y_true_)*(1.-distr[j](y_pred_)))
                    TP = tf.reduce_sum(y_true_*distr[j](y_pred_))
                    FP = tf.reduce_sum((1.-y_true_)*distr[j](y_pred_))
                    FN = tf.reduce_sum(y_true_*(1.-distr[j](y_pred_)))

                score_arr.append(score(TN,FP,FN,TP))

                if mode == 'weighted':
                    num_c0_arr.append(tf.cast(tf.shape(y_true_)[0],\
                                        tf.float32)-tf.reduce_sum(y_true_))

            score_arr = tf.stack(score_arr,axis=0)

            if mode == 'weighted':
                num_c0_arr = tf.stack(num_c0_arr,axis=0)
                final_score = tf.math.divide_no_nan(tf.reduce_sum\
                            (score_arr*num_c0_arr),tf.reduce_sum(num_c0_arr))

            if mode == 'average':
                final_score = tf.math.reduce_mean(score_arr)

            return -final_score + 1.

    return SOL_

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from matplotlib import pyplot as plt
from sklearn import metrics
import tensorflow as tf
import keras
from keras.layers import BatchNormalization
from keras.layers import Activation
from keras import optimizers
import math

def MSE(y_true, y_pred):
    return tf.reduce_mean(tf.math.square(y_true - y_pred))

import tensorflow as tf
def BCE(y_true, y_pred):
    return -tf.reduce_mean(y_true*tf.math.log(y_pred)+(1-y_true)*tf.math.log(1-y_pred))

def Ours_Accu(y_true, y_pred):
    y_pred = 1/(1+tf.math.exp(-L*(y_pred-0.5)))
    yl = y_train.shape[0]
    accu = (yl-tf.reduce_sum(y_true)-tf.reduce_sum(y_pred)+2*tf.reduce_sum(y_true*y_pred)) / yl
    return 1-accu

def Ours_Fbeta(y_true, y_pred):
    y_pred = 1/(1+tf.math.exp(-L*(y_pred-0.5)))
    numerator = (1+beta**2)*tf.reduce_sum(y_true*y_pred)
    denominator = (beta**2)*tf.reduce_sum(y_true) + tf.reduce_sum(y_pred)
    return 1-(numerator/denominator)

def Ours_Gmean(y_true, y_pred):
    y_pred = 1/(1+tf.math.exp(-L*(y_pred-0.5)))
    syhy = tf.reduce_sum(y_true*y_pred)
    sy = tf.reduce_sum(y_true)
    yl = (y_train.shape[0])
    gmean = tf.sqrt(syhy*(yl-tf.reduce_sum(y_pred)-sy+syhy)/(sy*(yl-sy)))
    return 1-gmean

def Ours_BAccu(y_true, y_pred):
    y_pred = 1/(1+tf.math.exp(-L*(y_pred-0.5)))
    syhy = tf.reduce_sum(y_true*y_pred)
    sy = tf.reduce_sum(y_true)
    yl = y_train.shape[0]
    baccu = (yl*(syhy+sy)-sy*(tf.reduce_sum(y_pred)+sy)) / (2*sy*(yl-sy))
    return 1-baccu

# **BREAST-CANCER DATA**

In [5]:
import pandas as pd
cancer_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Major-project-8th-sem/breast_cancer.csv')
cancer_df.shape


(569, 32)

In [6]:
cancer_df.isnull().sum()


,0
id,0
diagnosis,0
radius_mean,0
texture_mean,0
perimeter_mean,0
area_mean,0
smoothness_mean,0
compactness_mean,0
concavity_mean,0
concave points_mean,0


In [7]:
cancer_df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [8]:
cancer_df.describe()


,id,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
count,5.690000e+02,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,...,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000
mean,3.037183e+07,14.127292,19.289649,91.969033,654.889104,0.096360,0.104341,0.088799,0.048919,0.181162,...,16.269190,25.677223,107.261213,880.583128,0.132369,0.254265,0.272188,0.114606,0.290076,0.083946
std,1.250206e+08,3.524049,4.301036,24.298981,351.914129,0.014064,0.052813,0.079720,0.038803,0.027414,...,4.833242,6.146258,33.602542,569.356993,0.022832,0.157336,0.208624,0.065732,0.061867,0.018061
min,8.670000e+03,6.981000,9.710000,43.790000,143.500000,0.052630,0.019380,0.000000,0.000000,0.106000,...,7.930000,12.020000,50.410000,185.200000,0.071170,0.027290,0.000000,0.000000,0.156500,0.055040
25%,8.692180e+05,11.700000,16.170000,75.170000,420.300000,0.086370,0.064920,0.029560,0.020310,0.161900,...,13.010000,21.080000,84.110000,515.300000,0.116600,0.147200,0.114500,0.064930,0.250400,0.071460
50%,9.060240e+05,13.370000,18.840000,86.240000,551.100000,0.095870,0.092630,0.061540,0.033500,0.179200,...,14.970000,25.410000,97.660000,686.500000,0.131300,0.211900,0.226700,0.099930,0.282200,0.080040
75%,8.813129e+06,15.780000,21.800000,104.100000,782.700000,0.105300,0.130400,0.130700,0.074000,0.195700,...,18.790000,29.720000,125.400000,1084.000000,0.146000,0.339100,0.382900,0.161400,0.317900,0.092080
max,9.113205e+08,28.110000,39.280000,188.500000,2501.000000,0.163400,0.345400,0.426800,0.201200,0.304000,...,36.040000,49.540000,251.200000,4254.000000,0.222600,1.058000,1.252000,0.291000,0.663800,0.207500


In [9]:
y_encoded, y_class = pd.factorize(cancer_df['diagnosis'])
print(y_class)
y_encoded

Index(['M', 'B'], dtype='object')


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0,
       1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0,
       1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0,
       0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1,
       1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0,

In [10]:
y_encoded = (y_encoded+1)%2
y_encoded

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1,
       0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0,
       0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1,
       1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,

In [11]:
cancer_df['label'] = y_encoded
cancer_df = cancer_df.drop('id', axis=1)
cancer_df = cancer_df.drop('diagnosis', axis=1)
cancer_df

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,label
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890,1
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902,1
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758,1
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300,1
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,0.1726,0.05623,...,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115,1
565,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,0.1752,0.05533,...,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637,1
566,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,0.1590,0.05648,...,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820,1
567,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,0.2397,0.07016,...,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400,1


In [12]:
cancer_df.iloc[:,:-1] = (cancer_df.iloc[:,:-1] - cancer_df.iloc[:,:-1].mean())/cancer_df.iloc[:,:-1].std()
cancer_df

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,label
0,1.096100,-2.071512,1.268817,0.983510,1.567087,3.280628,2.650542,2.530249,2.215566,2.253764,...,-1.358098,2.301575,1.999478,1.306537,2.614365,2.107672,2.294058,2.748204,1.935312,1
1,1.828212,-0.353322,1.684473,1.907030,-0.826235,-0.486643,-0.023825,0.547662,0.001391,-0.867889,...,-0.368879,1.533776,1.888827,-0.375282,-0.430066,-0.146620,1.086129,-0.243675,0.280943,1
2,1.578499,0.455786,1.565126,1.557513,0.941382,1.052000,1.362280,2.035440,0.938859,-0.397658,...,-0.023953,1.346291,1.455004,0.526944,1.081980,0.854222,1.953282,1.151242,0.201214,1
3,-0.768233,0.253509,-0.592166,-0.763792,3.280667,3.399917,1.914213,1.450431,2.864862,4.906602,...,0.133866,-0.249720,-0.549538,3.391291,3.889975,1.987839,2.173873,6.040726,4.930672,1
4,1.748758,-1.150804,1.775011,1.824624,0.280125,0.538866,1.369806,1.427237,-0.009552,-0.561956,...,-1.465481,1.337363,1.219651,0.220362,-0.313119,0.612640,0.728618,-0.867590,-0.396751,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,2.109139,0.720838,2.058974,2.341795,1.040926,0.218868,1.945573,2.318924,-0.312314,-0.930209,...,0.117596,1.751022,2.013529,0.378033,-0.273077,0.663928,1.627719,-1.358963,-0.708467,1
565,1.703356,2.083301,1.614511,1.722326,0.102368,-0.017817,0.692434,1.262558,-0.217473,-1.057681,...,2.045599,1.420690,1.493644,-0.690623,-0.394473,0.236365,0.733182,-0.531387,-0.973122,1
566,0.701667,2.043775,0.672084,0.577445,-0.839745,-0.038646,0.046547,0.105684,-0.808406,-0.894800,...,1.373645,0.578492,0.427529,-0.808876,0.350427,0.326479,0.413705,-1.103578,-0.318129,1
567,1.836725,2.334403,1.980781,1.733693,1.524426,3.269267,3.294046,2.656528,2.135315,1.042778,...,2.235958,2.301575,1.651717,1.429169,3.901415,3.194794,2.287972,1.917396,2.217684,1


In [13]:
cancer_df['label'].value_counts()


,count
label,
0,357
1,212


In [14]:
df_0 = cancer_df[cancer_df['label']==0]
df_1 = cancer_df[cancer_df['label']==1]
print(len(df_0), len(df_1))

357 212


In [15]:
N = round(len(df_0)*0.1)
df_1_samp = df_1.sample(n=N, random_state = 100)
df_1_samp

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,label
219,1.533097,3.064460,1.482818,1.614345,-0.864631,0.163957,0.322388,0.449731,-1.399339,-1.369279,...,3.210535,2.170633,2.803894,0.369273,0.987914,0.610243,0.728618,-0.303481,-0.457654,1
45,1.283384,-0.392847,1.305856,1.196630,0.962713,1.216733,1.362280,1.339615,0.347926,-0.326840,...,-0.708923,1.289747,1.205600,1.556182,1.619046,2.216001,1.874173,1.451885,0.437632,1
408,1.096100,0.318610,1.063047,0.957083,0.514765,0.493423,0.392634,1.013350,0.657983,-0.298513,...,-0.043477,0.917752,0.822712,0.693374,0.757834,0.277588,1.259559,0.257396,0.060028,1
156,1.008133,0.337210,1.046586,0.877518,1.076478,1.176970,1.213007,1.455585,0.581381,-0.161126,...,-0.092288,0.763001,0.740163,0.413071,0.607202,0.412759,0.561273,-0.707570,-0.363530,1
213,0.934354,1.457870,0.927239,0.832905,0.301456,0.194253,0.995997,0.439423,-1.837067,-0.586034,...,0.389306,0.391006,0.246624,-0.353383,-0.476463,0.038881,-0.071597,-2.097638,-0.872908,1
100,-0.146789,1.323019,-0.161284,-0.205133,-0.105252,-0.364135,-0.031978,-0.103836,-0.739099,-0.578952,...,1.560751,0.039842,0.045520,-0.257029,-0.381126,0.214316,0.057716,-0.403695,-0.552332,1
372,2.055224,-0.974102,2.030166,2.077526,0.265905,0.892947,1.309595,1.973589,0.588676,-0.137048,...,-0.624319,1.334387,1.149396,-0.576750,0.188990,0.624144,1.247389,-0.276002,0.150276,1
212,3.967796,-0.190570,3.972634,5.240230,1.268455,0.894841,2.901420,2.849813,-0.596837,-1.069012,...,-1.172620,2.417638,2.842534,-0.795736,-0.652519,0.229655,0.682979,-2.024902,-1.588804,1
433,1.331624,0.623187,1.305856,1.293244,0.386780,0.654369,0.885610,0.992733,0.479244,-0.209282,...,0.854630,1.132021,1.268829,0.290438,0.584956,0.570459,0.854888,0.171729,-0.044616,1
365,1.791323,0.579012,1.721511,1.813257,-0.345580,0.165850,0.115287,0.745586,-0.706269,-1.023688,...,0.112715,1.605200,1.579706,0.014515,-0.105920,-0.009531,0.941603,-0.471582,-0.918862,1


In [16]:
cancer_df = concath(df_0, df_1_samp)
cancer_df

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,label
19,-0.166653,-1.146154,-0.185565,-0.251735,0.101657,-0.436466,-0.277965,-0.028584,0.267676,-0.727669,...,-1.044086,-0.225019,-0.297499,0.509425,-0.489175,-0.159083,0.215933,0.123238,-0.628739,0
20,-0.297184,-0.832276,-0.260877,-0.383301,0.792066,0.429044,-0.540886,-0.459223,0.566790,0.752425,...,-0.843964,-0.332451,-0.439238,-0.051181,0.148312,-0.398748,-0.635551,0.457824,-0.117147,0
21,-1.311926,-1.592558,-1.301661,-1.082620,0.429441,-0.746429,-0.743094,-0.725698,0.012334,0.885562,...,-1.629809,-1.253810,-0.993547,0.001375,-0.886413,-0.879660,-0.796202,-0.728583,-0.344152,0
37,-0.311372,-0.202195,-0.385162,-0.372503,-0.464322,-1.262592,-0.792517,-0.507415,-1.257077,-0.590283,...,-0.466499,-0.678556,-0.587827,-1.548613,-1.322484,-1.073022,-0.980890,-1.476957,-1.232240,0
46,-1.683090,-0.569549,-1.656820,-1.287215,-0.736646,-0.850381,-0.914695,-1.108221,-0.155461,0.316186,...,-0.604795,-1.488019,-1.121235,-0.116877,-0.753576,-0.974903,-1.353462,0.330132,-0.545688,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15,0.117112,1.918224,0.195933,0.011113,1.247125,1.044426,0.942058,0.637089,1.792428,1.129176,...,1.863374,0.501116,0.109978,1.551802,2.564154,2.063094,0.860973,2.129139,2.776892,1
489,0.727206,0.211659,0.622700,0.576024,-1.520911,-0.629033,-0.656165,-0.666166,0.125414,-1.352283,...,0.143628,0.596347,0.357275,-1.378241,0.239836,-0.117381,-0.414350,2.871047,-0.427202,1
451,1.550123,1.327669,1.470472,1.523414,0.486324,-0.106622,0.962128,1.074943,-0.542121,-1.258804,...,0.859511,0.968343,0.949171,0.894842,-0.443413,0.601615,0.486728,-0.982351,-1.275426,1
223,0.460467,0.223284,0.437507,0.302377,0.436552,0.304075,0.324896,0.404632,0.450062,0.032915,...,0.750502,0.554684,0.364300,0.999955,1.231342,0.601136,0.506505,1.765458,1.243223,1


In [17]:
cancer_df['label'].value_counts()


,count
label,
0,357
1,36


In [18]:
T_sse= []
T_mse= []
T_bce= []
T_acc= []
T_f1= []
T_f05= []
T_f2= []
T_gmean= []
T_bacc= []
T_sola = []
T_solf = []
T_solb = []

In [21]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state = 2)

X = cancer_df.iloc[:, :-1]
y = cancer_df.iloc[:, -1]
L = 73
hidden_node = 2
activation = 'sigmoid'
kernel_initializer=keras.initializers.he_normal(seed=100)
epochs=100

In [23]:
import numpy as np
n_iter=0
batch_size = int(X.shape[0]*0.9 * 0.05)
print('batch_size: ', batch_size)
learning_rate = 0.005

model = Sequential()
model.add(Dense(hidden_node, input_dim=X.shape[1], kernel_initializer=kernel_initializer))
model.add(BatchNormalization())
model.add(Activation(activation))
model.add(Dense(1))
opt = optimizers.Adam(learning_rate = learning_rate)

for train_index, test_index in skf.split(cancer_df, cancer_df['label']):
    n_iter += 1
    X_train = X.iloc[train_index]
    y_train= y.iloc[train_index]
    X_test = X.iloc[test_index]
    y_test= y.iloc[test_index]
    print('#'*50,'{0}th CV'.format(n_iter),'#'*50)
    X_train = np.array(X_train)
    y_train = np.array(y_train)
    y_train = y_train.astype(float)
    X_test = np.array(X_test)
    y_test = np.array(y_test)
    y_test = y_test.astype(float)

    start_mse = time.time()
    model.compile(loss=MSE, optimizer=opt, metrics=['accuracy'])
    history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, verbose=0, batch_size=batch_size)
    T_mse.append(time.time()-start_mse)
print(np.mean(T_mse))

batch_size:  17


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


################################################## 1th CV ##################################################
################################################## 2th CV ##################################################
################################################## 3th CV ##################################################
################################################## 4th CV ##################################################
################################################## 5th CV ##################################################
################################################## 6th CV ##################################################
################################################## 7th CV ##################################################
################################################## 8th CV ##################################################
################################################## 9th CV ##################################################
###################

In [24]:
n_iter=0

###################### MLP (sigmoid // sigmoid // BCE) ##############################
batch_size = int(X.shape[0]*0.9 * 0.05)
print('batch_size: ', batch_size)
learning_rate=0.005

model = Sequential()
model.add(Dense(hidden_node, input_dim=X.shape[1], kernel_initializer=kernel_initializer))
model.add(BatchNormalization())
model.add(Activation(activation))
model.add(Dense(1, activation='sigmoid'))
opt = optimizers.Adam(learning_rate = learning_rate)   # SGD(learning_rate=learning_rate, momentum=momentum)

for train_index, test_index in skf.split(cancer_df, cancer_df['label']):
    n_iter += 1
    X_train = X.iloc[train_index]
    y_train= y.iloc[train_index]
    X_test = X.iloc[test_index]
    y_test= y.iloc[test_index]
    print('#'*50,'{0}th CV'.format(n_iter),'#'*50)
    X_train = np.array(X_train)
    y_train = np.array(y_train)
    y_train = y_train.astype(float)
    X_test = np.array(X_test)
    y_test = np.array(y_test)
    y_test = y_test.astype(float)

    start_bce = time.time()
    model.compile(loss=BCE, optimizer=opt, metrics=['accuracy'])
    history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, verbose=0, batch_size=batch_size)
    T_bce.append(time.time()-start_bce)
print(np.mean(T_bce))

batch_size:  17
################################################## 1th CV ##################################################


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


################################################## 2th CV ##################################################
################################################## 3th CV ##################################################
################################################## 4th CV ##################################################
################################################## 5th CV ##################################################
################################################## 6th CV ##################################################
################################################## 7th CV ##################################################
################################################## 8th CV ##################################################
################################################## 9th CV ##################################################
################################################## 10th CV ##################################################
21.397303462028503

In [25]:
n_iter=0

batch_size = int(X.shape[0]*0.9 * 0.05)  # 0.05%0.004
print('batch_size: ', batch_size)
learning_rate = 0.005

model = Sequential()
model.add(Dense(hidden_node, input_dim=X.shape[1], kernel_initializer=kernel_initializer))
model.add(BatchNormalization())
model.add(Activation(activation))
model.add(Dense(1, activation='sigmoid'))
opt = optimizers.Adam(learning_rate = learning_rate)

for train_index, test_index in skf.split(cancer_df, cancer_df['label']):
    n_iter += 1
    X_train = X.iloc[train_index]
    y_train= y.iloc[train_index]
    X_test = X.iloc[test_index]
    y_test= y.iloc[test_index]
    print('#'*50,'{0}th CV'.format(n_iter),'#'*50)
    X_train = np.array(X_train)
    y_train = np.array(y_train)
    y_train = y_train.astype(float)
    X_test = np.array(X_test)
    y_test = np.array(y_test)
    y_test = y_test.astype(float)

    start_acc = time.time()
    model.compile(loss=Ours_Accu, optimizer=opt, metrics=['accuracy'])
    history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, verbose=0, batch_size=batch_size)
    T_acc.append(time.time()-start_acc)
print(np.mean(T_acc))

batch_size:  17
################################################## 1th CV ##################################################


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


################################################## 2th CV ##################################################
################################################## 3th CV ##################################################
################################################## 4th CV ##################################################
################################################## 5th CV ##################################################
################################################## 6th CV ##################################################
################################################## 7th CV ##################################################
################################################## 8th CV ##################################################
################################################## 9th CV ##################################################
################################################## 10th CV ##################################################
20.289661741256715

In [26]:
n_iter=0
batch_size = int(X.shape[0]*0.9 * 0.05)   # 0.05&0.001
print('batch_size: ', batch_size)
learning_rate=0.005

beta = 1
model = Sequential()
model.add(Dense(hidden_node, input_dim=X.shape[1], kernel_initializer=kernel_initializer))
model.add(BatchNormalization())
model.add(Activation(activation))
model.add(Dense(1, activation='sigmoid'))
opt = optimizers.Adam(learning_rate = learning_rate)

for train_index, test_index in skf.split(cancer_df, cancer_df['label']):
    n_iter += 1
    X_train = X.iloc[train_index]
    y_train= y.iloc[train_index]
    X_test = X.iloc[test_index]
    y_test= y.iloc[test_index]
    print('#'*50,'{0}th CV'.format(n_iter),'#'*50)
    X_train = np.array(X_train)
    y_train = np.array(y_train)
    y_train = y_train.astype(float)
    X_test = np.array(X_test)
    y_test = np.array(y_test)
    y_test = y_test.astype(float)

    start_f1 = time.time()
    model.compile(loss=Ours_Fbeta, optimizer=opt, metrics=['accuracy'])
    history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, verbose=0, batch_size=batch_size)
    T_f1.append(time.time()-start_f1)
print(np.mean(T_f1))

batch_size:  17
################################################## 1th CV ##################################################


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


################################################## 2th CV ##################################################
################################################## 3th CV ##################################################
################################################## 4th CV ##################################################
################################################## 5th CV ##################################################
################################################## 6th CV ##################################################
################################################## 7th CV ##################################################
################################################## 8th CV ##################################################
################################################## 9th CV ##################################################
################################################## 10th CV ##################################################
19.358650302886964

In [27]:
n_iter=0

batch_size = int(X.shape[0]*0.9 * 0.05)
print('batch_size: ', batch_size)
learning_rate=0.005

beta = 0.5
model = Sequential()
model.add(Dense(hidden_node, input_dim=X.shape[1], kernel_initializer=kernel_initializer))
model.add(BatchNormalization())
model.add(Activation(activation))
model.add(Dense(1, activation='sigmoid'))
opt = optimizers.Adam(learning_rate = learning_rate)

for train_index, test_index in skf.split(cancer_df, cancer_df['label']):
    n_iter += 1
    X_train = X.iloc[train_index]
    y_train= y.iloc[train_index]
    X_test = X.iloc[test_index]
    y_test= y.iloc[test_index]
    print('#'*50,'{0}th CV'.format(n_iter),'#'*50)
    X_train = np.array(X_train)
    y_train = np.array(y_train)
    y_train = y_train.astype(float)
    X_test = np.array(X_test)
    y_test = np.array(y_test)
    y_test = y_test.astype(float)

    start_f05 = time.time()
    model.compile(loss=Ours_Fbeta, optimizer=opt, metrics=['accuracy'])
    history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, verbose=0, batch_size=batch_size)
    T_f05.append(time.time()-start_f05)
print(np.mean(T_f05))

batch_size:  17
################################################## 1th CV ##################################################


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


################################################## 2th CV ##################################################
################################################## 3th CV ##################################################
################################################## 4th CV ##################################################
################################################## 5th CV ##################################################
################################################## 6th CV ##################################################
################################################## 7th CV ##################################################
################################################## 8th CV ##################################################
################################################## 9th CV ##################################################
################################################## 10th CV ##################################################
19.758426785469055

In [28]:
n_iter=0

batch_size = int(X.shape[0]*0.9 * 0.05)
print('batch_size: ', batch_size)
learning_rate=0.005

beta = 2
model = Sequential()
model.add(Dense(hidden_node, input_dim=X.shape[1], kernel_initializer=kernel_initializer))
model.add(BatchNormalization())
model.add(Activation(activation))
model.add(Dense(1, activation='sigmoid'))
opt = optimizers.Adam(learning_rate = learning_rate)

for train_index, test_index in skf.split(cancer_df, cancer_df['label']):
    n_iter += 1
    X_train = X.iloc[train_index]
    y_train= y.iloc[train_index]
    X_test = X.iloc[test_index]
    y_test= y.iloc[test_index]
    print('#'*50,'{0}th CV'.format(n_iter),'#'*50)
    X_train = np.array(X_train)
    y_train = np.array(y_train)
    y_train = y_train.astype(float)
    X_test = np.array(X_test)
    y_test = np.array(y_test)
    y_test = y_test.astype(float)

    start_f2 = time.time()
    model.compile(loss=Ours_Fbeta, optimizer=opt, metrics=['accuracy'])
    history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, verbose=0, batch_size=batch_size)
    T_f2.append(time.time()-start_f2)
print(np.mean(T_f2))

batch_size:  17
################################################## 1th CV ##################################################


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


################################################## 2th CV ##################################################
################################################## 3th CV ##################################################
################################################## 4th CV ##################################################
################################################## 5th CV ##################################################
################################################## 6th CV ##################################################
################################################## 7th CV ##################################################
################################################## 8th CV ##################################################
################################################## 9th CV ##################################################
################################################## 10th CV ##################################################
18.899096870422362

In [29]:
n_iter=0

batch_size = int(X.shape[0]*0.9 * 0.05)
print('batch_size: ', batch_size)
learning_rate=0.005

model = Sequential()
model.add(Dense(hidden_node, input_dim=X.shape[1], kernel_initializer=kernel_initializer))
model.add(BatchNormalization())
model.add(Activation(activation))
model.add(Dense(1, activation='sigmoid'))
opt = optimizers.Adam(learning_rate = learning_rate)

for train_index, test_index in skf.split(cancer_df, cancer_df['label']):
    n_iter += 1
    X_train = X.iloc[train_index]
    y_train= y.iloc[train_index]
    X_test = X.iloc[test_index]
    y_test= y.iloc[test_index]
    print('#'*50,'{0}th CV'.format(n_iter),'#'*50)
    X_train = np.array(X_train)
    y_train = np.array(y_train)
    y_train = y_train.astype(float)
    X_test = np.array(X_test)
    y_test = np.array(y_test)
    y_test = y_test.astype(float)

    start_gmean = time.time()
    model.compile(loss=Ours_Gmean, optimizer=opt, metrics=['accuracy'])
    history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, verbose=0, batch_size=batch_size)
    T_gmean.append(time.time()-start_gmean)
print(np.mean(T_gmean))

batch_size:  17
################################################## 1th CV ##################################################


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


################################################## 2th CV ##################################################
################################################## 3th CV ##################################################
################################################## 4th CV ##################################################
################################################## 5th CV ##################################################
################################################## 6th CV ##################################################
################################################## 7th CV ##################################################
################################################## 8th CV ##################################################
################################################## 9th CV ##################################################
################################################## 10th CV ##################################################
19.57393274307251


In [30]:
n_iter=0

batch_size = int(X.shape[0]*0.9 * 0.05)
print('batch_size: ', batch_size)
learning_rate=0.005

model = Sequential()
model.add(Dense(hidden_node, input_dim=X.shape[1], kernel_initializer=kernel_initializer))
model.add(BatchNormalization())
model.add(Activation(activation))
model.add(Dense(1, activation='sigmoid'))
opt = optimizers.Adam(learning_rate = learning_rate)

for train_index, test_index in skf.split(cancer_df, cancer_df['label']):
    n_iter += 1
    X_train = X.iloc[train_index]
    y_train= y.iloc[train_index]
    X_test = X.iloc[test_index]
    y_test= y.iloc[test_index]
    print('#'*50,'{0}th CV'.format(n_iter),'#'*50)
    X_train = np.array(X_train)
    y_train = np.array(y_train)
    y_train = y_train.astype(float)
    X_test = np.array(X_test)
    y_test = np.array(y_test)
    y_test = y_test.astype(float)

    start_bacc = time.time()
    model.compile(loss=Ours_BAccu, optimizer=opt, metrics=['accuracy'])
    history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, verbose=0, batch_size=batch_size)
    T_bacc.append(time.time()-start_bacc)
print(np.mean(T_bacc))

batch_size:  17
################################################## 1th CV ##################################################


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


################################################## 2th CV ##################################################
################################################## 3th CV ##################################################
################################################## 4th CV ##################################################
################################################## 5th CV ##################################################
################################################## 6th CV ##################################################
################################################## 7th CV ##################################################
################################################## 8th CV ##################################################
################################################## 9th CV ##################################################
################################################## 10th CV ##################################################
18.546088886260986

In [31]:
n_iter=0

batch_size = int(X.shape[0]*0.9 * 0.05)
print('batch_size: ', batch_size)
learning_rate=0.005

model = Sequential()
model.add(Dense(hidden_node, input_dim=X.shape[1], kernel_initializer=kernel_initializer))
model.add(BatchNormalization())
model.add(Activation(activation))
model.add(Dense(1, activation='sigmoid'))
opt = optimizers.Adam(learning_rate = learning_rate)

for train_index, test_index in skf.split(cancer_df, cancer_df['label']):
    n_iter += 1
    X_train = X.iloc[train_index]
    y_train= y.iloc[train_index]
    X_test = X.iloc[test_index]
    y_test= y.iloc[test_index]
    print('#'*50,'{0}th CV'.format(n_iter),'#'*50)
    X_train = np.array(X_train)
    y_train = np.array(y_train)
    y_train = y_train.astype(float)
    X_test = np.array(X_test)
    y_test = np.array(y_test)
    y_test = y_test.astype(float)

    start_sola = time.time()
    model.compile(loss=SOL(score = 'accuracy', distribution = 'cosine', mu = 0.5, delta = 0.1, mode = 'average'))
    history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, verbose=0, batch_size=batch_size)
    T_sola.append(time.time()-start_sola)
print(np.mean(T_sola))

batch_size:  17
################################################## 1th CV ##################################################


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


################################################## 2th CV ##################################################
################################################## 3th CV ##################################################
################################################## 4th CV ##################################################
################################################## 5th CV ##################################################
################################################## 6th CV ##################################################
################################################## 7th CV ##################################################
################################################## 8th CV ##################################################
################################################## 9th CV ##################################################
################################################## 10th CV ##################################################
19.48641369342804


In [32]:
times = []
names = [T_mse, T_bce, T_acc, T_f1, T_f05, T_f2, T_gmean, T_bacc, T_sola]
NAME = ['T_mse', 'T_bce', 'T_acc', 'T_f1', 'T_f05', 'T_f2', 'T_gmean', 'T_bacc', 'T_sola']
for i in range(len(names)):
    times.append(np.mean(names[i]))

rate = times/times[1]

df_time = pd.DataFrame(NAME, columns=['names'])
df_time['times'] = times
df_time['rate'] = rate
df_time

,names,times,rate
0,T_mse,21.301784,0.995536
1,T_bce,21.397303,1.000000
2,T_acc,20.289662,0.948235
3,T_f1,19.358650,0.904724
4,T_f05,19.758427,0.923407
5,T_f2,18.899097,0.883247
6,T_gmean,19.573933,0.914785
7,T_bacc,18.546089,0.866749
8,T_sola,19.486414,0.910695
